In [1]:
import time
import base64
from hashlib import md5
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import rsa, padding as asymmetric_padding
from cryptography.hazmat.primitives import serialization, hashes, padding as symmetric_padding
from cryptography.hazmat.backends import default_backend  

def generate_challenge(message, public_key):
    # Obtenemos el tiempo en microsegundos
    now = int(time.time()*1000000)
    # Este es un mal mecanismo para generar una clave
    key_seed = now.to_bytes(8,"big")
    key = md5(key_seed).digest()
    # Este es un mal mecanismo para generar un IV
    iv_seed = (now+1).to_bytes(8,"big")
    iv = md5(iv_seed).digest()
    # Ciframos el mensaje con AES-128 usando la clave `key`
    # Obtenemos una instancia del cifrador
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    # Usamos PKCS7 como mecanismo de relleno
    padder = symmetric_padding.PKCS7(128).padder()
    padded = padder.update(message) + padder.finalize()
    # Ciframos
    ciphertext = encryptor.update(padded) + encryptor.finalize()
    # Ciframos la clave secreta con la clave pública del destinatario
    encrypted_key = public_key.encrypt(
        key,
        asymmetric_padding.OAEP(
            mgf=asymmetric_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    assert(len(encrypted_key) == 128)
    # El desafío está compuesto por la clave cifrada, el iv y el texto cifrado
    return base64.b64encode(encrypted_key + iv + ciphertext)

# Clave pública serializada
pem = b'-----BEGIN PUBLIC KEY-----\nMIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQDIgCvlphhRoOt39+rBUxdqrivb\nxVdYmlGXkT/5mO6/fwYzA/yXeg1aKEIj1f1l1yXWecN0iYrDUflwyN5aMj6nH+zW\nYfMOVhp2seFya10vXA24UX5o5Ei7JLcnf1ubO3VfH4Fttcvvy27Qmkii3uedOTFi\nZtZn/U1dhm1rYhtrPwIDAQAB\n-----END PUBLIC KEY-----\n'
message = b"Este es el texto claro del mensaje"
# Deserializamos la clave pública
public_key = serialization.load_pem_public_key(pem)

generate_challenge(message, public_key)

b'lofJjow1dF3AEB6nDc9vytNe+jC5mLM4/iRUVBy/sZ3JaLFNMK3eCwQpyk0uQQhs/PlTa01ay0N5qoUIquQ6sEXZ2gkM9iymE7FlHljOnAf4pVuEUe75TkAW0faMeIFJ1zo/hXRydvDW0P7s1nKqPjfh2+IfiXWaQ+HkHC0GmOwnMBFlbq88ZIKUUEhRD6YdAULRZajOiS5PVqXY9gEfiBJI7NI0nq1PkOnoMG9+t+Sl2kr2NgzEEDnCaQjYfrbb'